In [4]:
TOTAL_COUNT = 100000
DB_NAME = "./100k_data_sqlite.db"

In [21]:
from tqdm import notebook as tqdm
import sqlite3
import random
from faker import Faker
# Connect to the SQLite database
conn = sqlite3.connect(DB_NAME)

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Display the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)


Tables in the database: [('Student',), ('Professor',), ('Course',), ('Department',), ('Enrollment',), ('Assignment',), ('Submission',), ('Schedule',), ('Major',), ('Textbook',), ('CourseTextbook',), ('Library',), ('Book',), ('BookLoan',), ('ResearchGroup',), ('ResearchProject',), ('ProjectMember',), ('Lab',), ('LabEquipment',), ('EquipmentMaintenance',)]


In [6]:


fake = Faker()
fake_u = Faker().unique


# Helper function to generate random date
def random_date(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())))

# Generate data for Department
departments = []
for i in tqdm.tqdm(range(TOTAL_COUNT), desc="Department"):
    department_id = i + 1
    name = fake.word().capitalize() + " Department"
    building = fake.street_name()
    departments.append((department_id, name, building))
cursor.executemany('INSERT INTO Department (department_id, name, building) VALUES (?, ?, ?);', departments)

# Generate data for Professor
professors = []
for i in tqdm.tqdm(range(TOTAL_COUNT), desc="Professor"):
    professor_id = i + 1
    name = fake.name()
    email = fake_u.email()
    department = random.choice(departments)[0]
    professors.append((professor_id, name, email, department))
cursor.executemany('INSERT INTO Professor (professor_id, name, email, department) VALUES (?, ?, ?, ?);', professors)

# Generate data for Course
courses = []
for i in tqdm.tqdm(range(TOTAL_COUNT), desc="Course"):
    course_id = i + 1
    title = fake.sentence(nb_words=3)
    credits = random.randint(1, 4)
    department = random.choice(departments)[0]
    courses.append((course_id, title, credits, department))
cursor.executemany('INSERT INTO Course (course_id, title, credits, department) VALUES (?, ?, ?, ?);', courses)

# Generate data for Student
students = []
for i in tqdm.tqdm(range(TOTAL_COUNT), desc="Student"):
    student_id = i + 1
    name = fake.name()
    email = fake_u.email()
    date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=30)
    major = random.choice(departments)[0]  # Assuming major is linked to department
    students.append((student_id, name, email, date_of_birth, major))
cursor.executemany('INSERT INTO Student (student_id, name, email, date_of_birth, major) VALUES (?, ?, ?, ?, ?);', students)

# Generate data for Enrollment
enrollments = []
for i in tqdm.tqdm(range(TOTAL_COUNT), desc="Enrollment"):
    enrollment_id = i + 1
    student_id = random.choice(students)[0]
    course_id = random.choice(courses)[0]
    semester = random.choice(['Spring', 'Summer', 'Fall', 'Winter']) + ' ' + str(random.randint(2019, 2023))
    grade = random.choice(['A', 'B', 'C', 'D', 'F', None])
    enrollments.append((enrollment_id, student_id, course_id, semester, grade))
cursor.executemany('INSERT INTO Enrollment (enrollment_id, student_id, course_id, semester, grade) VALUES (?, ?, ?, ?, ?);', enrollments)

# Commit changes and close the connection
conn.commit()

print("Data generation and insertion complete.")


  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

Data generation and insertion complete.


In [7]:
fake = Faker()


# Generate and insert data for remaining tables
assignments = []
submissions = []
schedules = []
majors = []
textbooks = []
course_textbooks = []
libraries = []
books = []
book_loans = []
research_groups = []
research_projects = []
project_members = []
labs = []
lab_equipments = []
equipment_maintenances = []

# Fetch necessary foreign keys from already populated tables
cursor.execute("SELECT student_id FROM Student")
student_ids = cursor.fetchall()
cursor.execute("SELECT professor_id FROM Professor")
professor_ids = cursor.fetchall()
cursor.execute("SELECT course_id FROM Course")
course_ids = cursor.fetchall()
cursor.execute("SELECT department_id FROM Department")
department_ids = cursor.fetchall()

# Generate data for each table
for i in tqdm.tqdm(range(TOTAL_COUNT)):
    assignment_id = i + 1
    course_id = random.choice(course_ids)[0]
    title = fake.sentence(nb_words=4)
    due_date = fake.date_between(start_date='today', end_date='+1y')
    assignments.append((assignment_id, course_id, title, due_date))

    schedule_id = i + 1
    professor_id = random.choice(professor_ids)[0]
    room = fake.building_number()
    time_slot = fake.time()
    schedules.append((schedule_id, course_id, professor_id, room, time_slot))

    major_id = i + 1
    name = fake.word().capitalize() + " Major"
    department_id = random.choice(department_ids)[0]
    majors.append((major_id, name, department_id))

    textbook_id = i + 1
    title = fake.sentence(nb_words=3)
    author = fake.name()
    isbn = fake_u.isbn13()
    textbooks.append((textbook_id, title, author, isbn))
    course_textbooks.append((course_id, textbook_id))

    library_id = i + 1
    name = fake.company() + " Library"
    location = fake.address()
    libraries.append((library_id, name, location))

    book_id = i + 1
    title = fake.sentence(nb_words=3)
    author = fake.name()
    isbn = fake_u.isbn13()
    books.append((book_id, title, author, isbn, library_id))

    loan_id = i + 1
    book_id = book_id
    student_id = random.choice(student_ids)[0]
    loan_date = fake.date_between(start_date='-1y', end_date='today')
    return_date = fake.date_between(start_date='today', end_date='+1y')
    book_loans.append((loan_id, book_id, student_id, loan_date, return_date))

    group_id = i + 1
    name = fake.word().capitalize() + " Research"
    focus_area = fake.sentence(nb_words=3)
    research_groups.append((group_id, name, focus_area))

    project_id = i + 1
    title = fake.sentence(nb_words=3)
    group_id = group_id
    start_date = fake.date_between(start_date='-1y', end_date='today')
    end_date = fake.date_between(start_date='today', end_date='+1y')
    research_projects.append((project_id, title, group_id, start_date, end_date))
    project_members.append((project_id, professor_id))

    lab_id = i + 1
    name = fake.company() + " Lab"
    building = fake.building_number()
    labs.append((lab_id, name, building))

    equipment_id = i + 1
    name = fake.word().capitalize() + " Equipment"
    lab_id = lab_id
    lab_equipments.append((equipment_id, name, lab_id))

    maintenance_id = i + 1
    equipment_id = equipment_id
    date = fake.date_between(start_date='-1y', end_date='today')
    details = fake.sentence(nb_words=6)
    equipment_maintenances.append((maintenance_id, equipment_id, date, details))

# Insert data into tables
cursor.executemany('INSERT INTO Assignment (assignment_id, course_id, title, due_date) VALUES (?, ?, ?, ?);', assignments)
cursor.executemany('INSERT INTO Schedule (schedule_id, course_id, professor_id, room, time_slot) VALUES (?, ?, ?, ?, ?);', schedules)
cursor.executemany('INSERT INTO Major (major_id, name, department_id) VALUES (?, ?, ?);', majors)
cursor.executemany('INSERT INTO Textbook (textbook_id, title, author, isbn) VALUES (?, ?, ?, ?);', textbooks)
cursor.executemany('INSERT INTO CourseTextbook (course_id, textbook_id) VALUES (?, ?);', course_textbooks)
cursor.executemany('INSERT INTO Library (library_id, name, location) VALUES (?, ?, ?);', libraries)
cursor.executemany('INSERT INTO Book (book_id, title, author, isbn, library_id) VALUES (?, ?, ?, ?, ?);', books)
cursor.executemany('INSERT INTO BookLoan (loan_id, book_id, student_id, loan_date, return_date) VALUES (?, ?, ?, ?, ?);', book_loans)
cursor.executemany('INSERT INTO ResearchGroup (group_id, name, focus_area) VALUES (?, ?, ?);', research_groups)
cursor.executemany('INSERT INTO ResearchProject (project_id, title, group_id, start_date, end_date) VALUES (?, ?, ?, ?, ?);', research_projects)
cursor.executemany('INSERT INTO ProjectMember (project_id, professor_id) VALUES (?, ?);', project_members)
cursor.executemany('INSERT INTO Lab (lab_id, name, building) VALUES (?, ?, ?);', labs)
cursor.executemany('INSERT INTO LabEquipment (equipment_id, name, lab_id) VALUES (?, ?, ?);', lab_equipments)
cursor.executemany('INSERT INTO EquipmentMaintenance (maintenance_id, equipment_id, date, details) VALUES (?, ?, ?, ?);', equipment_maintenances)
conn.commit()


  0%|          | 0/100000 [00:00<?, ?it/s]

代码漏的 submissions

In [8]:
# Fetching foreign key data from Assignment and Student tables
cursor.execute("SELECT assignment_id FROM Assignment")
assignments = cursor.fetchall()

cursor.execute("SELECT student_id FROM Student")
students = cursor.fetchall()

# Convert fetched data into more usable formats (lists of ids)
assignment_ids = [assignment[0] for assignment in assignments]
student_ids = [student[0] for student in students]

# Display the fetched data to verify
print("Assignment IDs:", assignment_ids)
print("Student IDs:", student_ids)


Assignment IDs: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 2

In [22]:
import random
from datetime import datetime, timedelta

# Function to generate random dates
def random_date(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())))

# Define the date range for the submissions
start_date = datetime.strptime('2022-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2022-12-31', '%Y-%m-%d')

# Generate random data for the Submission table
submission_data = []
for _ in tqdm.tqdm(range(random.randint(TOTAL_COUNT, TOTAL_COUNT*1.2))):
    submission_id = len(submission_data) + 1
    assignment_id = random.choice(assignment_ids)
    student_id = random.choice(student_ids)
    submission_date = random_date(start_date, end_date)
    grade = random.choice(['A', 'B', 'C', 'D', 'F', 'I'])  # Including 'I' for Incomplete

    submission_data.append((submission_id, assignment_id, student_id, submission_date, grade))

# Insert data into the Submission table
insert_query = "INSERT INTO Submission (submission_id, assignment_id, student_id, submission_date, grade) VALUES (?, ?, ?, ?, ?)"
cursor.executemany(insert_query, submission_data)
conn.commit()

print(f"{len(submission_data)} records inserted into Submission table.")


  0%|          | 0/118906 [00:00<?, ?it/s]

IntegrityError: UNIQUE constraint failed: Submission.submission_id

In [19]:
conn.close()

print("Data generation and insertion for all tables complete.")

Data generation and insertion for all tables complete.


In [17]:
# cursor.execute("DELETE FROM Submission WHERE submission_id != -1")
# conn.commit()